# Datensammlung und -vorbereitung
Dieses Notebook sammelt die Wetter-, Zeit- und MOER-Daten und bereitet diese für die Prognose vor:


*   Laden der Wetterdaten aus Copernicus Datei
*   Laden der MOER Werte für DE und NO
*   Kombinieren zu einem Datensatz
*   Hinzufügen von Zeitvariablen
*   Setzen des Datums als Index
*   Überprüfung des Datensatzes auf Vollständigkeit
*   Filtern der Daten für separaten Datensatz ohne Ausreißer





In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np
import pytz
from google.colab import drive
import glob
import os
import pytz
import holidays
from datetime import datetime
from pathlib import Path

In [ ]:
drive.mount('/content/drive')
base_path = '/content/drive/My Drive/data_collection/'
weather_data_path = '/content/drive/My Drive/data_collection/copernicus/dataset-sis-energy-derived-reanalysis/'
start_date = '2021-01-01'
end_date = '2024-03-01'
countries=['DE', 'NO']

Mounted at /content/drive


Load weather data

In [ ]:
def load_and_process_data(file_name, start_date, end_date, variable_prefix):
    df = pd.read_csv(f'{weather_data_path}{file_name}', skiprows=52)
    df = df.rename(columns={'Date':'date'})
    df['date'] = pd.to_datetime(df['date'])
    df.set_index('date', inplace=True)
    df = df.loc[start_date:end_date]
    filtered_columns = [col for col in df.columns if any(country in col for country in countries)]
    df = df[filtered_columns]
    df.columns = [variable_prefix + col for col in df.columns]
    return df

ghi_df = load_and_process_data("H_ERA5_ECMW_T639_GHI_0000m_Euro_INS_TIM_01h.csv", start_date, end_date, 'ghi_')
temperature_df = load_and_process_data("H_ERA5_ECMW_T639_TA-_0002m_Euro_INS_TIM_01h.csv", start_date, end_date, 'temperature_')
wind_speed_df = load_and_process_data("H_ERA5_ECMW_T639_WS-_0100m_Euro_INS_TIM_01h.csv", start_date, end_date, 'wind_speed_')
precipitation_df = load_and_process_data("H_ERA5_ECMW_T639_TP-_0000m_Euro_ACC_TIM_01h.csv", start_date, end_date, 'precipitation_')

temperature_df = temperature_df  - 273.15  # Convert from K to C

Load MOER DE

In [ ]:
moer_de_df = pd.DataFrame()
moer_files_de = glob.glob(os.path.join(base_path, 'WattTime/moer_DE_*.csv'))

for file in moer_files_de:
    df = pd.read_csv(file, index_col='date')
    df['country'] = 'DE'
    moer_de_df = pd.concat([moer_de_df, df])

# moer_de_df.to_csv(base_path + 'moer_DE_202210_202402.csv', index=True)
moer_de_df

,moer,country
date,,
2022-10-01 00:00:00,785.209331,DE
2022-10-01 01:00:00,752.652765,DE
2022-10-01 02:00:00,789.344578,DE
2022-10-01 03:00:00,786.490729,DE
2022-10-01 04:00:00,786.589007,DE
...,...,...
2024-01-31 19:00:00,784.521383,DE
2024-01-31 20:00:00,789.990947,DE
2024-01-31 21:00:00,799.501259,DE


Load MOER NO

In [ ]:
moer_no_df = pd.DataFrame()
moer_files_no = glob.glob(os.path.join(base_path, 'WattTime/moer_NO_*.csv'))

for file in moer_files_no:
    df = pd.read_csv(file, index_col='date')
    df['country'] = 'NO'
    moer_no_df = pd.concat([moer_no_df, df])

# moer_no_df.to_csv(base_path + 'moer_NO_202210_202402.csv', index=True)
moer_no_df

,moer,country
date,,
2021-01-01 00:00:00,203.133617,NO
2021-01-01 01:00:00,222.963148,NO
2021-01-01 02:00:00,226.198771,NO
2021-01-01 03:00:00,225.673360,NO
2021-01-01 04:00:00,211.577988,NO
...,...,...
2024-02-29 19:00:00,273.156882,NO
2024-02-29 20:00:00,262.905703,NO
2024-02-29 21:00:00,258.838495,NO


Combine MOER and weather

In [ ]:
# Combine moer data of DE and NO
moer_combined_df = pd.concat([moer_de_df, moer_no_df])
# moer_combined_df.to_csv(base_path + 'moer_DE_NO.csv')
moer_combined_df.index = pd.DatetimeIndex(moer_combined_df.index)

In [ ]:
# Combine weather and moer data
weather_moer_df = pd.DataFrame()

for country in countries:
    moer_country_df = moer_combined_df[moer_combined_df['country'] == country]
    for data_type in ['temperature', 'ghi', 'wind_speed', 'precipitation']:
        weather_df = locals()[f"{data_type}_df"]
        moer_country_df = moer_country_df.join(weather_df, how='left')

    # Concatenate the results
    weather_moer_df = pd.concat([weather_moer_df, moer_country_df.reset_index()], ignore_index=True)

weather_moer_df

,date,moer,country,temperature_DE,temperature_NO,ghi_DE,ghi_NO,wind_speed_DE,wind_speed_NO,precipitation_DE,precipitation_NO
0,2022-10-01 00:00:00,785.209331,DE,8.16702,4.57775,6.467518e-14,6.467518e-14,7.456859,5.949002,2.802370e-05,0.000133
1,2022-10-01 01:00:00,752.652765,DE,8.15188,4.58203,6.467518e-14,6.467518e-14,7.849637,6.010181,1.064189e-04,0.000156
2,2022-10-01 02:00:00,789.344578,DE,8.12730,4.69708,6.467518e-14,6.467518e-14,8.228340,6.070655,2.747895e-04,0.000202
3,2022-10-01 03:00:00,786.490729,DE,8.29520,4.76748,6.467518e-14,6.467518e-14,8.638514,6.191992,4.385244e-04,0.000271
4,2022-10-01 04:00:00,786.589007,DE,8.37817,4.91440,6.467518e-14,6.467518e-14,9.047430,6.259928,5.098294e-04,0.000369
...,...,...,...,...,...,...,...,...,...,...,...
40123,2024-02-29 19:00:00,273.156882,NO,5.89474,1.85858,-6.467518e-14,-6.467518e-14,6.461082,7.893519,6.660060e-06,0.000139
40124,2024-02-29 20:00:00,262.905703,NO,5.29388,1.85190,-6.467518e-14,-6.467518e-14,6.363908,7.709262,3.929645e-06,0.000164
40125,2024-02-29 21:00:00,258.838495,NO,4.82380,1.77972,-6.467518e-14,-6.467518e-14,6.209530,7.530479,2.435606e-06,0.000178
40126,2024-02-29 22:00:00,256.014885,NO,4.28008,1.80856,-6.467518e-14,-6.467518e-14,5.666669,7.141570,1.030581e-06,0.000193


In [ ]:
# Rename columns
weather_moer_df['temperature'] = np.where(weather_moer_df['country'] == 'DE', weather_moer_df['temperature_DE'], weather_moer_df['temperature_NO'])
weather_moer_df['ghi'] = np.where(weather_moer_df['country'] == 'DE', weather_moer_df['ghi_DE'], weather_moer_df['ghi_NO'])
weather_moer_df['wind_speed'] = np.where(weather_moer_df['country'] == 'DE', weather_moer_df['wind_speed_DE'], weather_moer_df['wind_speed_NO'])
weather_moer_df['precipitation'] = np.where(weather_moer_df['country'] == 'DE', weather_moer_df['precipitation_DE'], weather_moer_df['precipitation_NO'])

weather_moer_df.drop(['temperature_DE', 'temperature_NO', 'ghi_DE', 'ghi_NO', 'wind_speed_DE', 'wind_speed_NO', 'precipitation_DE', 'precipitation_NO' ], axis=1, inplace=True)
weather_moer_df = weather_moer_df[['date', 'country', 'ghi', 'precipitation', 'temperature', 'wind_speed', 'moer']]
weather_moer_df.sort_values(by='date', inplace=True)

weather_moer_df

,date,country,ghi,precipitation,temperature,wind_speed,moer
12408,2021-01-01 00:00:00,NO,0.000000e+00,4.596687e-05,-4.46052,3.224764,203.133617
12409,2021-01-01 01:00:00,NO,0.000000e+00,4.844980e-05,-4.58634,3.212784,222.963148
12410,2021-01-01 02:00:00,NO,0.000000e+00,4.715853e-05,-4.63306,3.245318,226.198771
12411,2021-01-01 03:00:00,NO,0.000000e+00,4.323623e-05,-4.69846,3.278241,225.673360
12412,2021-01-01 04:00:00,NO,0.000000e+00,4.257162e-05,-4.78565,3.293060,211.577988
...,...,...,...,...,...,...,...
11661,2024-02-29 21:00:00,DE,-6.467518e-14,2.435606e-06,4.82380,6.209530,793.305948
40126,2024-02-29 22:00:00,NO,-6.467518e-14,1.927261e-04,1.80856,7.141570,256.014885
11662,2024-02-29 22:00:00,DE,-6.467518e-14,1.030581e-06,4.28008,5.666669,797.868328
11663,2024-02-29 23:00:00,DE,-6.467518e-14,5.369242e-07,3.91686,5.609017,790.845212


Add time variables

In [ ]:
weather_moer_df['date'] = pd.to_datetime(weather_moer_df['date'])
weather_moer_df['hour_of_day'] = weather_moer_df['date'].dt.hour
weather_moer_df['day_of_year'] = weather_moer_df['date'].dt.dayofyear
weather_moer_df['day_of_week'] = weather_moer_df['date'].dt.weekday


def is_holiday_or_weekend(date, country_code):
    if date in holidays.country_holidays(country_code):
        return 1
    if date.weekday() >= 5:
        return 1
    return 0

def get_season(date):
    year = date.year
    spring_start = datetime(year, 3, 20)
    summer_start = datetime(year, 6, 21)
    autumn_start = datetime(year, 9, 23)
    winter_start = datetime(year, 12, 21)

    if spring_start <= date < summer_start:
        return '0'
    elif summer_start <= date < autumn_start:
        return '1'
    elif autumn_start <= date < winter_start:
        return '2'
    else:
        return '3'

weather_moer_df['is_holiday_or_weekend'] = weather_moer_df.apply(lambda row: is_holiday_or_weekend(row['date'], row['country']), axis=1)
weather_moer_df['season'] = weather_moer_df['date'].apply(get_season)

In [ ]:
# Set index
weather_moer_df.set_index('date', inplace=True)
weather_moer_df.index = pd.DatetimeIndex(weather_moer_df.index)

In [ ]:
# Check completeness
start_date = weather_moer_df.index.min()
end_date = weather_moer_df.index.max()
full_range = pd.date_range(start=start_date, end=end_date, freq='H')
missing_times = full_range.difference(weather_moer_df.index)

if missing_times.empty:
    print("No data missing")
else:
    print("Missing data points:", missing_times)

No data missing


In [ ]:
# Save final dataset
weather_time_moer_df = weather_moer_df[['country', 'ghi', 'precipitation', 'temperature', 'wind_speed', 'hour_of_day', 'day_of_week', 'day_of_year', 'is_holiday_or_weekend', 'season', 'moer']]
weather_time_moer_df.to_csv(f'{base_path}weather_time_moer_2021_2024_DE_NO.csv', index=True)
weather_time_moer_df

,country,ghi,precipitation,temperature,wind_speed,hour_of_day,day_of_week,day_of_year,is_holiday_or_weekend,season,moer
date,,,,,,,,,,,
2021-01-01 00:00:00,NO,0.000000e+00,4.596687e-05,-4.46052,3.224764,0,4,1,1,3,203.133617
2021-01-01 01:00:00,NO,0.000000e+00,4.844980e-05,-4.58634,3.212784,1,4,1,1,3,222.963148
2021-01-01 02:00:00,NO,0.000000e+00,4.715853e-05,-4.63306,3.245318,2,4,1,1,3,226.198771
2021-01-01 03:00:00,NO,0.000000e+00,4.323623e-05,-4.69846,3.278241,3,4,1,1,3,225.673360
2021-01-01 04:00:00,NO,0.000000e+00,4.257162e-05,-4.78565,3.293060,4,4,1,1,3,211.577988
...,...,...,...,...,...,...,...,...,...,...,...
2024-02-29 21:00:00,DE,-6.467518e-14,2.435606e-06,4.82380,6.209530,21,3,60,0,3,793.305948
2024-02-29 22:00:00,NO,-6.467518e-14,1.927261e-04,1.80856,7.141570,22,3,60,0,3,256.014885
2024-02-29 22:00:00,DE,-6.467518e-14,1.030581e-06,4.28008,5.666669,22,3,60,0,3,797.868328


### Filter outliers

In [ ]:
weather_time_moer_df=pd.read_csv(f'{base_path}weather_time_moer_2021_2024_DE_NO.csv', index_col='date')
weather_time_moer_df

,country,ghi,precipitation,temperature,wind_speed,hour_of_day,day_of_week,day_of_year,is_holiday_or_weekend,season,moer
date,,,,,,,,,,,
2021-01-01 00:00:00,NO,0.000000e+00,4.596687e-05,-4.46052,3.224764,0,4,1,1,3,203.133617
2021-01-01 01:00:00,NO,0.000000e+00,4.844980e-05,-4.58634,3.212784,1,4,1,1,3,222.963148
2021-01-01 02:00:00,NO,0.000000e+00,4.715853e-05,-4.63306,3.245318,2,4,1,1,3,226.198771
2021-01-01 03:00:00,NO,0.000000e+00,4.323623e-05,-4.69846,3.278241,3,4,1,1,3,225.673360
2021-01-01 04:00:00,NO,0.000000e+00,4.257162e-05,-4.78565,3.293060,4,4,1,1,3,211.577988
...,...,...,...,...,...,...,...,...,...,...,...
2024-02-29 21:00:00,DE,-6.467518e-14,2.435606e-06,4.82380,6.209530,21,3,60,0,3,793.305948
2024-02-29 22:00:00,NO,-6.467518e-14,1.927261e-04,1.80856,7.141570,22,3,60,0,3,256.014885
2024-02-29 22:00:00,DE,-6.467518e-14,1.030581e-06,4.28008,5.666669,22,3,60,0,3,797.868328


In [ ]:
moer_de = weather_time_moer_df[weather_time_moer_df['country'] == "DE"][['moer']]
moer_no = weather_time_moer_df[weather_time_moer_df['country'] == "NO"][['moer']]

Define bounds for Germany



In [ ]:
Q1 = moer_de['moer'].quantile(0.25)
Q3 = moer_de['moer'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

Count outliers Germany

In [ ]:
outliers_count = moer_de[(moer_de['moer'] < lower_bound) | (moer_de['moer'] > upper_bound)].shape[0]
outliers_count

882

Filter outliers Germany

In [ ]:
weather_time_moer_df_filtered = weather_time_moer_df[
    ~((weather_time_moer_df['country'] == 'DE') &
      ((weather_time_moer_df['moer'] <= lower_bound) |
       (weather_time_moer_df['moer'] > upper_bound)))
]

In [ ]:
moer_de_filtered.describe()

,moer
count,11526.000000
mean,765.561895
std,24.682715
min,687.717291
25%,749.933103
50%,765.234274
75%,785.112943
max,839.795349


Define bounds for Norway

In [ ]:
Q1 = moer_no['moer'].quantile(0.25)
Q3 = moer_no['moer'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

Count outliers Norway

In [ ]:
outliers_count = moer_no[(moer_no['moer'] < lower_bound) | (moer_no['moer'] > upper_bound)].shape[0]
outliers_count

2630

Filter outliers Norway

In [ ]:
weather_time_moer_df_filtered = weather_time_moer_df_filtered[
    ~((weather_time_moer_df_filtered['country'] == 'NO') &
      ((weather_time_moer_df_filtered['moer'] <= lower_bound) |
       (weather_time_moer_df_filtered['moer'] > upper_bound)))
]

In [ ]:
moer_no_filtered.describe()

,moer
count,25090.000000
mean,237.046259
std,25.845394
min,166.978555
25%,220.877569
50%,234.977666
75%,251.297528
max,312.097756


Save filtered dataframe

In [ ]:
weather_time_moer_df_filtered.to_csv(f'{base_path}weather_time_moer_filtered_2021_2024_DE_NO.csv', index=True)
weather_time_moer_df_filtered

,country,ghi,precipitation,temperature,wind_speed,hour_of_day,day_of_week,day_of_year,is_holiday_or_weekend,season,moer
date,,,,,,,,,,,
2021-01-01 00:00:00,NO,0.000000e+00,4.596687e-05,-4.46052,3.224764,0,4,1,1,3,203.133617
2021-01-01 01:00:00,NO,0.000000e+00,4.844980e-05,-4.58634,3.212784,1,4,1,1,3,222.963148
2021-01-01 02:00:00,NO,0.000000e+00,4.715853e-05,-4.63306,3.245318,2,4,1,1,3,226.198771
2021-01-01 03:00:00,NO,0.000000e+00,4.323623e-05,-4.69846,3.278241,3,4,1,1,3,225.673360
2021-01-01 04:00:00,NO,0.000000e+00,4.257162e-05,-4.78565,3.293060,4,4,1,1,3,211.577988
...,...,...,...,...,...,...,...,...,...,...,...
2024-02-29 21:00:00,DE,-6.467518e-14,2.435606e-06,4.82380,6.209530,21,3,60,0,3,793.305948
2024-02-29 22:00:00,NO,-6.467518e-14,1.927261e-04,1.80856,7.141570,22,3,60,0,3,256.014885
2024-02-29 22:00:00,DE,-6.467518e-14,1.030581e-06,4.28008,5.666669,22,3,60,0,3,797.868328
